In [ ]:
import random as r

def actor_picker(l1):
    actor=r.choice(l1)
    return actor



In [ ]:

def Theme_picker(l2):
    Theme=r.choice(l2)
    return Theme





In [26]:
l1=["New York","Bali","Us","Dubai","karnataka"]
def location_picker(l3):
    location=r.choice(l3)
    return location





In [27]:
import os
import configparser

from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere

from langchain_core.messages import HumanMessage, SystemMessage

config = configparser.ConfigParser()
config.read('/workspaces/langchainCDAC9nov/config.ini')
groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')

messages = [
    SystemMessage(content='You are a Story Creator'),
    HumanMessage(content='please create a story related to tiger?')
]

## code for cohere.
model = ChatCohere(model="command-r-plus")
print(model.invoke(messages))

## Code for Groq
model = ChatGroq(model="llama3-8b-8192")
print(model.invoke(messages))

content="Once upon a time, in a dense and lush forest, there lived a majestic tiger named Raja. He was known for his striking orange and black stripes, powerful build, and intense gaze. Raja was the king of his domain, ruling over the other animals with strength and dignity.\n\nRaja had a proud and solitary nature, often spending his days patrolling his territory and ensuring the safety of the creatures under his watch. He was a fierce hunter, but only took what he needed to survive, respecting the balance of nature. The other animals both feared and respected him, knowing that he was a force to be reckoned with.\n\nOne day, Raja's peaceful reign was disrupted by the arrival of a group of humans who ventured into the forest. They were loud and destructive, cutting down trees and disturbing the peaceful habitat. Raja watched them with suspicion, knowing that their presence could threaten the delicate balance of his kingdom.\n\nAmong the humans was a young girl named Maya. She was advent

In [52]:
#The classes used for setting up the prompt

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate 

parser = StrOutputParser()
l1=["akshay Kumar","Priyaka chopara","Dipika Padukone","Allu Arjun","Sai pallavi","Mahesh babu"]
l2=["Dramatic","Horror","Romantic","silent","Comedy"]
l3=["New York","Bali","Us","Dubai","karnataka"]
actor= actor_picker(l1) # Based on user input pick a puzzle.
theme=Theme_picker(l2)
location=location_picker(l3)
writer=[actor,theme,location]
# templatized system prompt
system_template = "create a story by using following actor, theme and location. Please provide a {responseType} response." 

# Create prompt template instance.
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", writer)
    ]
)


# prompt Template also implements runnable and can be easily chained.
model = ChatGroq(model="llama3-8b-8192")
chain = prompt_template | model | parser
chain.invoke({"responseType":"brief"})



'Here\'s a brief story:\n\nTitle: The High-Stakes Heist\n\nAkshay Kumar plays the role of Rohan, a former thief turned security expert, who is lured back into the world of crime by a mysterious offer from a wealthy Dubai-based businessman. The businessman, known only as "The Sultan," promises Rohan a share of his vast fortune if he can pull off a daring heist in the luxurious city of Dubai.\n\nRohan agrees, and with the help of his old gang, they plan to steal a priceless diamond necklace from a high-end jewelry store in the heart of Dubai\'s financial district. As they execute their plan, they are pursued by a determined detective, who is determined to catch them in the act.\n\nAs the stakes get higher, Rohan finds himself in a game of cat and mouse, where he must use all his wits and skills to outsmart his pursuers and get away with the loot. But as the night wears on, he begins to realize that The Sultan may not be what he seems, and that the true prize may be more than just the dia

In [ ]:
from langchain_core.runnables import RunnableSequence
chain = RunnableSequence(model, parser)
chain.invoke([HumanMessage(content="hi i am bob")])

In [56]:
# import the chat history classes
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
import asyncio # library for writing code that interacts with DB, network calls etc. 

#Create a store in memory
store = InMemoryChatMessageHistory()


# Lets define a function that gets messages from store
async def getMessage():
    await asyncio.sleep(2) # this will mimic a read from DB
    print("Messages retrieved from DB")
    return await store.aget_messages()

# Now lets first add the first message to the store
store.add_message(HumanMessage('Hi! I am writer'))

messages = await(getMessage())


response = model.invoke(messages) # asyncio has runners for coroutines, context managers etc. 
print(response.content) # note that our first message is safely in the store

# lets add the message returned by the model to the store

store.add_message(SystemMessage(response.content))


store.add_message(HumanMessage('Lets see if you know my previous story?'))

messages = await(getMessage())

print(messages) # check all the message are in store.

response = model.invoke(messages)

print(response.content) # Notice that the reponse now takes into account earlier interactions also.

Messages retrieved from DB
Hello! Nice to meet you! As a writer, you must be excited to share your stories and ideas with the world. What kind of writing do you do? Fiction, non-fiction, poetry, or something else? I'm all ears!
Messages retrieved from DB
[HumanMessage(content='Hi! I am writer', additional_kwargs={}, response_metadata={}), SystemMessage(content="Hello! Nice to meet you! As a writer, you must be excited to share your stories and ideas with the world. What kind of writing do you do? Fiction, non-fiction, poetry, or something else? I'm all ears!", additional_kwargs={}, response_metadata={}), HumanMessage(content='Lets see if you know my previous story?', additional_kwargs={}, response_metadata={})]
I apologize, but I'm a large language model, I don't have the ability to retain information or recall previous conversations. Each time you interact with me, it's a new conversation and I don't have any prior knowledge or context. If you'd like to share your story with me, I'd b